In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 66.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c60f272cbd1d925e1f6dee54e811619b393a41998bed3a94938a3650580d9d54
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').\
        appName('alex').getOrCreate()
        
sc = spark.sparkContext

sc.addFile('/content/drive/MyDrive/transactions_amostra.csv')

# carregando o arquivo
rdd = sc.textFile('file://' + SparkFiles.get('transactions_amostra.csv'))

# fazendo split de sentenças em palavras
palavras = rdd.map(lambda frase: frase.split(';')).filter(lambda l: l[1].isnumeric())

In [4]:
# Questão1 
# 1. (Easy) The number of transactions involving Brazil;
questao1 = palavras.filter(lambda x: 'Brazil' in x[0]).map(lambda x: (x[0], 1)).reduceByKey(lambda x ,y: x + y)
questao1.take(5)

questao1.coalesce(1).saveAsTextFile('questao1.txt')


In [5]:
# Questão2
# 2. (Easy) The number of transactions per year;
questao2 = palavras.map(lambda x: (x[1], 1)).reduceByKey(lambda x ,y: x + y)
questao2.take(5)

questao2.coalesce(1).saveAsTextFile('questao2.txt')

In [6]:
# Questão3 
# 3. (Easy) The number of transactions per flow type and year;
questao3 = palavras.map(lambda x: (x[1] + " " + x[4], 1)).reduceByKey(lambda x ,y: x + y)
questao3.take(5)

questao3.coalesce(1).saveAsTextFile('questao3.txt')

In [7]:
#Questão 4
# 4. (Easy) The average of commodity values per year;
questao4 = palavras.map(lambda x: (x[1], float(x[5]))).groupByKey()\
.map(lambda x: (x[0], sum(list(x[1])) / len(list(x[1]))))

questao4.collect()
questao4.saveAsTextFile('questao4.txt')

In [8]:
# Questão 5
# (Easy) The average price of commodities per unit type, year, and category in the export flow in Brazil;
questao5 = palavras.filter(lambda x: x[0] == "Brazil" and x[4] == "Export")
finalQT5 = questao5.map(lambda x: ((x[1] + " " + x[7] + " " + x[9]), float(x[5]))).groupByKey()\
.map(lambda x: (x[0], sum(list(x[1])) / len(list(x[1]))))

finalQT5.take(10)
finalQT5.saveAsTextFile('questao5.txt')

In [9]:
# Questao 6 
# (Medium) The maximum, minimum, and mean transaction price per unit type and year;
questao6 = palavras.map(lambda x: ((x[7], x[1]), float(x[5]))).groupByKey().map(lambda x: (x[0], [min(x[1]), max(x[1]), sum(x[1]) / len(x[1])]))
questao6.take(1)

questao6.saveAsTextFile('questao6.txt')

In [10]:
# Questao 7
# (Hard) The most commercialized commodity (summing the quantities) in 2016, per flow type.

questao7 = palavras.filter(lambda x: '2016' in x[1]).map(lambda x: ((x[3], x[4]), float(x[8]))).groupByKey().map(lambda x: (x[0][1], (sum(list(x[1])), x[0][0]))).reduceByKey(lambda x, y: x if x > y else y)
questao7.take(4)

questao7.saveAsTextFile('questao7.txt')